In [8]:
import numpy as np 
import matplotlib.pyplot as plt

In [9]:
def initialise(layer_sizes, activation='relu', seed=42):
    np.random.seed(seed)
    num_layers = len(layer_sizes) - 1

    # Initialize weights and biases using Xavier initialisation
    weights = []
    biases = []

    for i in range(num_layers):
        weight_matrix = np.random.randn(layer_sizes[i], layer_sizes[i+1]) * np.sqrt(2 / (layer_sizes[i] + layer_sizes[i+1]))
        bias_vector = np.zeros((1, layer_sizes[i+1]))
        weights.append(weight_matrix)
        biases.append(bias_vector)
    
    print(f"Initialized network with layer sizes: {layer_sizes}")
    print(f"Activation function: {activation}")
    print(f"Number of layers: {num_layers}")
    return weights, biases, layer_sizes, activation

In [10]:
def count_parameters(weights, biases):
    total_params = 0
    for w, b in zip(weights, biases):
        total_params += w.size + b.size
    return total_params

Activation functiosn with their respective derivative

In [11]:
def relu(x):
    return np.maximum(0, x)
def relu_derivative(x):
    return (x > 0).astype(float)

def tanh(x):
    return np.tanh(x)
def tanh_derivative(x):
    return 1 - np.tanh(x)**2

def sigmoid(x):
    return 1 / (1 + np.exp(-x))

def sigmoid_derivative(x):
    sig = sigmoid(x)
    return sig * (1 - sig)

def softmax(x):
    exp_x = np.exp(x - np.max(x, axis=1, keepdims=True))
    return exp_x / np.sum(exp_x, axis=1, keepdims=True)

def apply_activation(x, activation_type):
    if activation_type == 'relu':
        return relu(x)
    elif activation_type == 'tanh':
        return tanh(x)
    elif activation_type == 'sigmoid':
        return sigmoid(x)
    else:
        raise ValueError("Unsupported activation function: {activation_type}")

Forward propagation throughout the network

mathematical formulation: 
    For each layer *l*: 
    Z[l] = A[l-1] @ W[l] + b[l] (Linear transformation)
    A[l] = activation(Z[l]) (Activation Function)

    Final Layer uses Softmax for classifcation

    Args: 
        X: Input data (batch_size, input_features)
        return_cache: If true return immediate values for back prop

    returns: 
        Predictions: Final output probaility (batch_size, num_classes)
        cache: Dictionary of intermediate values (if return_cache=True)

In [12]:
def forward_pass(X, weights, biases, activation_type, return_cache=False):
    cache = {}
    A = X
    cache['A0'] = X # Store input as activation of layer 0

    num_layers = len(weights)

    for i in range(num_layers):
        # Hidden Layers
        Z = np.dot(A, weights[i]) + biases[i]
        
        if i < num_layers - 1:
            A = apply_activation(Z, activation_type)
        else:
            A = softmax(Z)  # Output layer with softmax
        
        cache[f'Z{i+1}'] = Z
        cache[f'A{i+1}'] = A

    if return_cache:
        return A, cache
    return A

def predict(X, weights, biases, activation_type):
    probabilities = forward_pass(X, return_cache=False)
    return np.argmax(probabilities, axis=1)

In [13]:
def testing_forward_pass():
    print("="*50)
    print("Testing forward pass")
    print("="*50)

    X = np.array([[0,0],[0,1],[1,0],[1,1]], dtype=np.float32)
    y = np.array([0,1,1,0], dtype=int)  # XOR truth table

    weights, biases, layer_sizes, activation = initialise(layer_sizes=[2, 2, 2], activation='relu')

    print("\nNetwork Architecture:")
    for i in range(len(layer_sizes)-1):
        print(f" Layer {i+1}: {layer_sizes[i]} -> {layer_sizes[i+1]}")

    # forward pass
    print(f"\n{'-'*30}")
    print("FORWARD PASS:")
    print(f"{'-'*30}")

    predictions, cache = forward_pass(X, weights, biases, activation, return_cache=True)
    predicted_classes = np.argmax(predictions, axis=1)

    print(f"\nFinal Results:")
    print(f" Predicted Probabilities:\n{predictions}")
    print(f" Predicted Classes: {predicted_classes}")
    print(f" True Classes: {y}")
    print(f"Accuracy: {np.mean(predicted_classes == y) * 100:.2f}%")

    return weights, biases, layer_sizes, activation, X, y, cache

Loss Function

In [15]:
def softmax(x):

    '''
    Compute the softmax activation for a vector or a matrix of logits.

    Args: 
        x: A numpy array of shape (n,) or (m, n) where n is the number of classes.
    Returns:
        A numpy array of the same shape as x representing the softmax probabilities.
    '''

    shifted_x = x - np.max(x, axis=-1, keepdims=True)  # Prevent overflow
    exp_x = np.exp(shifted_x)

    return exp_x / np.sum(exp_x, axis=-1, keepdims=True)

In [16]:
def cross_entropy_loss(y_true, y_pred):

    '''
    Compute the cross-entropy loss between true labels and predicted probabilities.
    Mathematical Formula:
    L = -1/N * Σ(i=1 to N) Σ(j=1 to C) y_true[i,j] * log(y_pred[i,j])

    Args:
        y_true: A numpy array of shape (m, n) representing one-hot encoded true labels.
        y_pred: A numpy array of shape (m, n) representing predicted probabilities.
    Returns:
        A float representing the average cross-entropy loss over the batch.
    '''
    batch_size = y_pred.shape[0]
    num_classes = y_pred.shape[1]

    if y_true.ndim == 1:
        y_true_onehot = np.zeros((batch_size, num_classes))
        y_true_onehot[np.arange(batch_size), y_true] = 1
    else:
        y_true_onehot = y_true

    epsilon = 1e-15
    y_pred_clipped = np.clip(y_pred, epsilon, 1 - epsilon)

    # Calclulate cross-entropy loss
    sample_losses = -np.sum(y_true_onehot * np.log(y_pred_clipped), axis=1)

    return np.mean(sample_losses), y_true_onehot

In [36]:
def backward_pass(cache, y_true_onehot, weights, activation_type):
    """
    Backward propagation through the network
    
    Mathematical Formulation:
    
    For output layer (L):
    dZ[L] = A[L] - y_true  (gradient after softmax + cross-entropy)
    dW[L] = (1/m) * A[L-1].T @ dZ[L]
    db[L] = (1/m) * sum(dZ[L], axis=0)
    
    For hidden layers (l = L-1, L-2, ..., 1):
    dA[l] = dZ[l+1] @ W[l+1].T
    dZ[l] = dA[l] * activation_derivative(Z[l])
    dW[l] = (1/m) * A[l-1].T @ dZ[l]
    db[l] = (1/m) * sum(dZ[l], axis=0)
    
    Args:
        cache: Dictionary containing forward pass intermediate values
        y_true_onehot: One-hot encoded true labels (batch_size, num_classes)
        weights: List of weight matrices
        activation_type: Type of activation function used in hidden layers
    
    Returns:
        gradients: Dictionary containing gradients for weights and biases
    """
    
    num_layers = len(weights)
    batch_size = y_true_onehot.shape[0]
    
    # Initialize gradients lists with correct size
    dW_gradients = [None] * num_layers
    db_gradients = [None] * num_layers
    
    print(f"Starting backpropagation...")
    print(f"Batch size: {batch_size}, Number of layers: {num_layers}")
    
    # === OUTPUT LAYER GRADIENTS ===
    # For softmax + cross-entropy, the gradient simplifies to: dZ = A - y_true
    A_output = cache[f'A{num_layers}']  # Final predictions (after softmax)
    dZ = A_output - y_true_onehot  # Shape: (batch_size, num_classes)
    
    print(f"Output layer gradient dZ shape: {dZ.shape}")
    
    # Gradients for output layer weights and biases (last layer index)
    A_prev = cache[f'A{num_layers-1}']  # Activations from previous layer
    dW = (1/batch_size) * A_prev.T @ dZ  # Shape: (prev_layer_size, num_classes)
    db = (1/batch_size) * np.sum(dZ, axis=0, keepdims=True)  # Shape: (1, num_classes)
    
    # Store output layer gradients at correct index
    dW_gradients[num_layers-1] = dW
    db_gradients[num_layers-1] = db
    
    print(f"Output layer (index {num_layers-1}): dW shape = {dW.shape}, db shape = {db.shape}")
    
    # === HIDDEN LAYER GRADIENTS ===
    # Work backwards from layer (L-1) to layer 0
    for layer_idx in range(num_layers - 2, -1, -1):  # num_layers-2, num_layers-3, ..., 0
        # Propagate error backwards: dA = dZ_next @ W_next.T
        next_layer_idx = layer_idx + 1
        dA = dZ @ weights[next_layer_idx].T  # Shape: (batch_size, current_layer_size)
        
        # Get pre-activation values and compute activation derivative
        Z_current = cache[f'Z{layer_idx+1}']  # Pre-activation values (cache uses 1-based indexing)
        activation_derivative = apply_activation(Z_current, activation_type)
        
        # Apply chain rule: dZ = dA * activation_derivative
        dZ = dA * activation_derivative  # Element-wise multiplication
        
        # Get activations from previous layer
        A_prev = cache[f'A{layer_idx}']  # Previous layer activations
        
        # Calculate weight and bias gradients
        dW = (1/batch_size) * A_prev.T @ dZ
        db = (1/batch_size) * np.sum(dZ, axis=0, keepdims=True)
        
        # Store gradients at correct index
        dW_gradients[layer_idx] = dW
        db_gradients[layer_idx] = db
        
        print(f"Layer {layer_idx}: dZ shape = {dZ.shape}, dW shape = {dW.shape}, db shape = {db.shape}")
    
    gradients = {'dW': dW_gradients, 'db': db_gradients}
    
    print("Backpropagation complete!")
    return gradients

In [37]:
def update_weights(weights, biases, gradients, learning_rate):
    """
    Update weights and biases using gradients
    
    Mathematical Update Rule:
    W_new = W_old - learning_rate * dW
    b_new = b_old - learning_rate * db
    
    Args:
        weights: List of current weight matrices
        biases: List of current bias vectors
        gradients: Dictionary containing gradients
        learning_rate: Step size for gradient descent
    
    Returns:
        Updated weights and biases (in-place modification)
    """
    num_layers = len(weights)
    for i in range(num_layers):
        weights[i] -= learning_rate * gradients['dW'][i]
        biases[i] -= learning_rate * gradients['db'][i]
    return weights, biases

In [41]:
def train_one_step(X, y_true, weights, biases, activation_type, learning_rate=0.01):
    """
    Complete training step: forward pass + backward pass + weight update
    
    Args:
        X: Input data (batch_size, input_features)
        y_true: True labels (batch_size,)
        weights: List of weight matrices
        biases: List of bias vectors
        activation_type: Activation function for hidden layers
        learning_rate: Learning rate for gradient descent
    
    Returns:
        loss: Current loss value
        accuracy: Current accuracy percentage
        weights: Updated weights
        biases: Updated biases
    """
    # Forward pass
    predictions, cache = forward_pass(X, weights, biases, activation_type, return_cache=True)
    
    # Calculate loss
    loss, y_true_onehot = cross_entropy_loss(y_true, predictions)
    
    # Calculate accuracy
    predicted_classes = np.argmax(predictions, axis=1)
    accuracy = np.mean(predicted_classes == y_true) * 100
    
    # Backward pass
    gradients = backward_pass(cache, y_true_onehot, weights, activation_type)
    
    # Update weights
    weights, biases = update_weights(weights, biases, gradients, learning_rate)
    
    return loss, accuracy, weights, biases

In [39]:
def test_backpropagation_xor():
    """Test backpropagation on XOR problem"""
    print("="*60)
    print("TESTING BACKPROPAGATION ON XOR PROBLEM")
    print("="*60)
    
    # XOR dataset
    X = np.array([[0, 0],
                  [0, 1], 
                  [1, 0],
                  [1, 1]], dtype=np.float32)
    y = np.array([0, 1, 1, 0])
    
    print(f"XOR Dataset:")
    print(f"X:\n{X}")
    print(f"y: {y}")
    
    # Initialize network
    weights, biases, layer_sizes, activation = initialise([2, 4, 2], 'relu', seed=42)
    
    print(f"\nInitial network architecture: {layer_sizes}")
    print(f"Activation: {activation}")
    
    # Train for a few steps
    learning_rate = 0.1
    num_steps = 10
    
    print(f"\nTraining for {num_steps} steps with learning_rate = {learning_rate}")
    print("-" * 60)
    
    for step in range(num_steps):
        loss, accuracy, weights, biases = train_one_step(
            X, y, weights, biases, activation, learning_rate
        )
        
        if step % 2 == 0 or step == num_steps - 1:  # Print every 2nd step
            print(f"Step {step+1:2d}: Loss = {loss:.6f}, Accuracy = {accuracy:5.1f}%")
    
    # Final test
    print("\n" + "="*60)
    print("FINAL RESULTS")
    print("="*60)
    
    final_predictions = forward_pass(X, weights, biases, activation)
    predicted_classes = np.argmax(final_predictions, axis=1)
    final_accuracy = np.mean(predicted_classes == y) * 100
    
    print(f"Final predictions:\n{final_predictions}")
    print(f"Predicted classes: {predicted_classes}")
    print(f"True classes:      {y}")
    print(f"Final accuracy: {final_accuracy:.1f}%")
    
    print(f"\nImprovement: Network should be learning to solve XOR!")
    if final_accuracy > 75:
        print("The network is learning")
    else:
        print("Try more training steps or different learning rate")
    
    return weights, biases

In [43]:
if __name__ == "__main__":
    trained_weights, trained_biases = test_backpropagation_xor()
    print("\n" + "="*60)
    print("BACKPROPAGATION IMPLEMENTATION COMPLETE!")
    print("="*60)
    print("Key functions implemented:")
    print("- backward_pass(): Calculates gradients")
    print("- update_weights(): Updates parameters using gradients") 
    print("- train_one_step(): Complete training iteration")

TESTING BACKPROPAGATION ON XOR PROBLEM
XOR Dataset:
X:
[[0. 0.]
 [0. 1.]
 [1. 0.]
 [1. 1.]]
y: [0 1 1 0]
Initialized network with layer sizes: [2, 4, 2]
Activation function: relu
Number of layers: 2

Initial network architecture: [2, 4, 2]
Activation: relu

Training for 10 steps with learning_rate = 0.1
------------------------------------------------------------
Starting backpropagation...
Batch size: 4, Number of layers: 2
Output layer gradient dZ shape: (4, 2)
Output layer (index 1): dW shape = (4, 2), db shape = (1, 2)
Layer 0: dZ shape = (4, 4), dW shape = (2, 4), db shape = (1, 4)
Backpropagation complete!
Step  1: Loss = 0.719186, Accuracy =  75.0%
Starting backpropagation...
Batch size: 4, Number of layers: 2
Output layer gradient dZ shape: (4, 2)
Output layer (index 1): dW shape = (4, 2), db shape = (1, 2)
Layer 0: dZ shape = (4, 4), dW shape = (2, 4), db shape = (1, 4)
Backpropagation complete!
Starting backpropagation...
Batch size: 4, Number of layers: 2
Output layer gradie